# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np 
import pandas as pd 
import requests
import xlsxwriter
import math 

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [2]:
stocks = pd.read_csv("sp_500_stocks.csv")
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [3]:
from secretsiex import IEX_CLOUD_API_TOKEN

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [4]:
symbol = 'AAPL'
api_url = f'https://cloud.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

{'avgTotalVolume': 50982436, 'calculationPrice': 'tops', 'change': 0.33, 'changePercent': 0.00202, 'close': None, 'closeSource': 'official', 'closeTime': None, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': None, 'delayedPriceTime': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPrice': None, 'extendedPriceTime': None, 'high': None, 'highSource': None, 'highTime': None, 'iexAskPrice': 164.1, 'iexAskSize': 100, 'iexBidPrice': 162.5, 'iexBidSize': 2501, 'iexClose': 164.1, 'iexCloseTime': 1682520736696, 'iexLastUpdated': 1682520736696, 'iexMarketPercent': 0.01478891669081867, 'iexOpen': 163.01, 'iexOpenTime': 1682515800506, 'iexRealtimePrice': 164.1, 'iexRealtimeSize': 6, 'iexVolume': 189311, 'lastTradeTime': 1682520736696, 'latestPrice': 164.1, 'latestSource': 'IEX real time price', 'latestTime': '10:52:16 AM', 'latestUpdate': 1682520736696, 'latestVolume': None, 'low': None, 'lowSource': None, 'lowTime': None, 'marketCap': 2596381338600, 'oddLotDel

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [5]:
price = data['latestPrice']
market_cap = data['marketCap']
print(market_cap/1000000000000)

2.5963813386


## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [6]:
my_columns = ["Ticker", "Stock Price", "Market Capitalization", "Number of Shares to Buy"]
final_dataframe = pd.DataFrame(columns = my_columns)

In [7]:
final_dataframe = pd.concat(
        [
            final_dataframe, 
            pd.Series(
                [
                    symbol, 
                    price, 
                    market_cap, 
                    'N/A'
                ], 
                index = my_columns, 
                ).to_frame().T,
        ]
    )
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,164.1,2596381338600,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [8]:
final_dataframe = pd.DataFrame(columns=my_columns)
for stock in stocks['Ticker']:
    api_url = f'https://cloud.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    final_dataframe = pd.concat(
        [
            final_dataframe, pd.Series(
                [
                    stock,
                    data['latestPrice'],
                    data['marketCap'],
                    'N/A'
                ],
                index=my_columns,
            ).to_frame().T
        ]
    )

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [9]:
final_dataframe=final_dataframe.reset_index(drop=True)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,132.28,39115441115,N/A
1,AAL,12.72,8303825766,N/A
2,AAP,123.98,7348763368,N/A
3,AAPL,164.09,2596223119140,N/A
4,ABBV,160.99,284009186289,N/A
...,...,...,...,...
130,DG,219.5,48094310702,N/A
131,DGX,146.93,16438213970,N/A
132,DHI,106.92,36467329710,N/A
133,DHR,232.77,169714178896,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [11]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [26]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#    print(symbol_strings[i])
final_dataframe = pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        if symbol == 'HFC' or symbol == 'VIAC' or symbol == 'WLTW' or symbol == 'DISCA':
            continue
        final_dataframe = pd.concat(
            [
                final_dataframe, pd.Series(
                    [
                        symbol,
                        data[symbol]['quote']['latestPrice'],
                        data[symbol]['quote']['marketCap'],
                        'N/A'
                    ],
                    index=my_columns,
                ).to_frame().T
            ]
        )
final_dataframe=final_dataframe.reset_index(drop=True)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,132.7,39239635893,N/A
1,AAL,12.87,8401748239,N/A
2,AAP,123.81,7338686826,N/A
3,AAPL,164.815,2607694029990,N/A
4,ABBV,162.25,286232004941,N/A
...,...,...,...,...
496,YUM,138.23,38719309902,N/A
497,ZBH,138.52,28947945061,N/A
498,ZBRA,284.13,14604094758,N/A
499,ZION,27,3998674566,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

## Saving Our Excel Output

Saving our Excel file is very easy: